In [1]:
# https://peterbloem.nl/blog/transformers

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [173]:
torch.manual_seed(123123)
tokens_amount = 100
heads_num = 4
head_size = 64
emb_len = heads_num * head_size
ctx_len = 400
batch_size = 32

emb = nn.Embedding(tokens_amount, emb_len)
wq = torch.randn(heads_num,emb_len,head_size)
wk = torch.randn(heads_num,emb_len,head_size)
wv = torch.randn(heads_num,emb_len,head_size)

x = torch.randint(0, tokens_amount, (batch_size, ctx_len))

print(f'{wq.shape=}')
print(f'{x.shape=}')

wq.shape=torch.Size([4, 256, 64])
x.shape=torch.Size([32, 400])


In [174]:
xe = emb(x)
print(f'{xe.shape=}')

xe.shape=torch.Size([32, 400, 256])


In [178]:
xq = torch.einsum('bti,hic->bhtc',xe,wq)
xk = torch.einsum('bti,hic->bhtc',xe,wk)
xv = torch.einsum('bti,hic->bhtc',xe,wv)

print(f'{xq.shape=}')

xq.shape=torch.Size([32, 4, 400, 64])


In [164]:
wij = torch.einsum('bti,bTi->btT',xq,xk) / (emb_len ** 0.5)
wij = F.softmax(wij,dim=2)
wij

tensor([[[0.3478, 0.3622, 0.2900],
         [0.3539, 0.3723, 0.2738],
         [0.3142, 0.3156, 0.3702]],

        [[0.3274, 0.3222, 0.3503],
         [0.3454, 0.3776, 0.2770],
         [0.2808, 0.2604, 0.4588]]], grad_fn=<SoftmaxBackward0>)

In [187]:
y1 = wij @ xv
y2 = torch.einsum('bti,bic->btc',wij,xv)
(y1 == y2).all()

RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1